In [1]:
import math
from pathlib import Path

import numpy as np
from foambryo import infer_forces
from foambryo.dcel import DcelData
from foambryo.iorec import load_rec

fname = "../data/Meshes/Benchmark_original_mesh.rec"

points,triangles, labels = load_rec(fname)
triangles_and_labels = np.hstack((triangles, labels))
mesh = DcelData(points, triangles_and_labels)
dict_tensions, dict_pressure = infer_forces(mesh)

reference_tensions = {}
with Path("reference_tensions.txt").open("r") as f:
    for line in f.readlines():
        l1, l2, tension = line.split(" ")
        reference_tensions[(int(l1), int(l2))] = float(tension)

assert reference_tensions == dict_tensions

reference_pressures = {}
with Path("reference_pressures.txt").open("r") as f:
    for line in f.readlines():
        cell_id, pressure = line.split(" ")
        reference_pressures[int(cell_id)] = float(pressure)

assert reference_pressures.keys() == dict_pressure.keys()
# pressures use stochastic gradient descent at some point so there's some randomness
for cell_id in reference_pressures:
    assert math.fabs(reference_pressures[cell_id] - dict_pressure[cell_id]) < 0.0001

print("OK !")


OK !
